# Hyffix on testbed

You need to load hyffix IPython extension to activate `hylang`, `deffix`, and `runtests` magics.

`%hylang` / `%%hylang` magics are used for evaluating Hy code inside the Jupyter Notebook cell. `%hylang` refers to a single line magic, `%%hylang` to a multiline magic.        

`%deffix` / `%%deffix` magics are similar to `hylang` macros, but they will also import `deffix` macros which can be used to handle infix, prefix, and postfix notated Hy code. You could do same with `%%hylang` and then using require and import to load hyffix library.

`%runtests` magic collects all `test_` prefixed functions from namespace and then runs functions assuming that they contain test assertions. Tests can be written in Python or in Hy because functions on both languages are collected from the very same Jupyter Notebook user namespace.

In [1]:
%load_ext hyffix

Let us test a simple assertion inside a test function, first in Python and then in Hy:

In [2]:
def test_this_is_a_python_test_function():
  "this is a Python test function"
  assert 1 == 0

In [3]:
%%hylang
(defn test_this_is_a_hy_test_function []
  "this is a Hy test function"
  (assert (= 1 1)))

In [4]:
%runtests

Next we want to try some Hy code with deffix macros support:

In [5]:
%%deffix

(defmacro defassert [a f b]
  `(assert (~f ~a ~b)))

(defn test_assert []
  "Test assert"
  (defassert (deffix (1 + 1)) = (+ 1 2)))

In [6]:
%runtests newonly

## Archimedes tests

Finally I have installed [Archimedes](https://github.com/tuturto/archimedes) test suite and [Hamcrest](https://pypi.python.org/pypi/PyHamcrest) with `$ pip install libarchimedes pyhamcrest`. Then I'm ready to run these example tests from the Archimedes [readme](https://github.com/tuturto/archimedes/blob/master/README.rst) file:

In [7]:
%%hylang

;# pip install libarchimedes
;# pip install pyhamcrest
(require [archimedes [background fact check defmatcher attribute-matcher
                      assert-macro-error assert-error assert-right
                      with-background]])

(import [hypothesis.strategies [integers]]
        [hamcrest [assert-that]]
        [math [pow]])

(fact "true is always true"
      (assert True))

(background some-numbers
      a 3
      b 4
      c 5)

(fact "Pythagorean theorem holds in this specific case"
      (with-background some-numbers [a b c]
        (assert (= (+ (pow a 2) (pow b 2)) (pow c 2)))))

(fact "sum of two positive numbers is larger than either one of them"
      (variants :a (integers :min-value 1)
                :b (integers :min-value 1))
      (assert (> (+ a b) a))
      (assert (> (+ a b) b)))

(fact "example can clarify things"
      (variants :a (integers :min-value 0 :max-value 10)
                :b (integers :min-value 0 :max-value 10))
      (sample :a 0 :b 0)
      (assert (<= 0 (+ a b) 20)))

(fact "profile controls test settings"
      (variants :a (integers :min-value 0))
      (profile :max-examples 500)
      (assert (<= 0 a)))

(fact "macro errors can be asserted"
      (assert-macro-error "cond branches need to be a list"
                          (cond (= 1 1) True)))

(fact "even fact can be asserted for macro errors"
      (assert-macro-error "too many variants forms"
                          (fact "I'm incorrect"
                                (variants :a (integers))
                                (variants :a (integers))
                                (assert (= a a)))))

(fact "errors can be asserted"
      (assert-error "error"
                    (raise (ValueError "error"))))

(check "this is executed immediately"
       (assert (= 1 1)))

(defmatcher is-zero? []
            :match? (= item 0)
            :match! "a zero"
            :no-match! (.format "was a value of {0}" item))

(assert-that 0 (is-zero?))

(attribute-matcher item-with-length?
                   len =
                   "an item with length {0}"
                   "was an item with length {0}")

;(assert-that "foo" (is- (item-with-length? 3)))

Ok: this is executed immediately


In [8]:
%runtests newonly

Test function name,Status
Successful,8
Failed,1
Errors,0
Execution,0.5589 seconds
test_errors_can_be_asserted,successful
test_profile_controls_test_settings,successful
test_macro_errors_can_be_asserted,successful
test_sum_of_two_positive_numbers_is_larger_than_either_one_of_them,successful
test_true_is_always_true,successful
test_example_can_clarify_things,successful


In [9]:
from IPython.core.display import HTML
with open('ipynbtester_style.css') as f:
    css = f.read()
HTML('<style>%s</style>' % css)

## The [MIT](http://choosealicense.com/licenses/mit/) License

Copyright (c) 2017 Marko Manninen